# Empirical Risk Minimization

## Risk minimization

We assume that $\mathbf{x}$ and $y$ come from the distribution $\mathcal{X} \times \mathcal{Y}$. The training data $\mathcal{D} = \{(\mathbf{x}_i, y_i)\}_{i \in [n]}$ used to train the model consists of $n$ realizations from the distribution $\mathcal{X} \times \mathcal{Y}$. In supervised learning, recall that the goal is to find a predictor $f$ that predicts the response value given a previously unseen instance $\mathbf{x}$ sampled from $\mathcal{X}$. Of course, we want the model to be accurate in the sense that $f(\mathbf{x})$ is "close to" $y$ on average. We can measure the accuracy by introducing a loss function $\ell: \mathcal{Y}\times \mathcal{Y}\rightarrow \mathbb{R}$, whose value $\ell(f(\mathbf{x}), y)$ is large if $f$ is a poor predictor. In practice, many loss function can be used depending on the context, some examples include

1. Square loss (regression): $\ell(y, z)=(y-z)^2$
2. Absolute loss (regression): $\ell(y, z)=|y-z|$
2. Zero-one loss (classification): $\ell(y, z)= \mathbb{1}\{y\neq z\}$

We want to find a predictor $f$ such that $\ell(f(\mathbf{x}), y)$ is small on average. This can be framed as an optimization problem:

$$f^* = \underset{f}{\text{argmin}}\; \underbrace{\mathbb{E}_{(\mathbf{x}, y)\sim \mathcal{X}\times \mathcal{Y}}[\ell(f(\mathbf{x}), y)]}_{R(f)} \tag{1}$$

<i>Note that expectation is included since $\mathbf{x}, y$ are random variables.</i>

The term $R(f)$ is said to be the risk, or the expected loss, associated with the predictor $f$. The solution $f^*$ to $(1)$ is called the Bayes optimal predictor. Note that, by the law of total expectation, we can decompose the risk as

$$
\begin{align*}
    R(f) &= \mathbb{E}_{(\mathbf{x}, y)}[\ell(f(\mathbf{x}), y)]\\
    &=  \mathbb{E}_{\mathbf{x}}[\mathbb{E}_{y|\mathbf{x}}[\ell(f(\mathbf{x}), y)|\mathbf{x}=x]]
\end{align*}
$$

Which, under some regularity condition, the solution $f^*$ satisfies

$$f^*(x) = \underset{f}{\text{argmin}} \; \mathbb{E}_{y|\mathbf{x}}[\ell(f(\mathbf{x}), y)|\mathbf{x}=x]=\underset{z\in \mathcal{Y}}{\text{argmin}} \; \mathbb{E}_{y|\mathbf{x}}[\ell(z, y)|\mathbf{x}=x]\tag{2}$$

This simplifies the optimization problem $(1)$, since instead of optimizing over a family of functions, we are now optimizing over $\mathcal{Y}$, which is usually a nicer set (for example, $\mathbb{R}$ for regression problems and a finite set for classification problems). For certain choices of loss functions, the solution admits a simple form, as shown in the following example. <br>


```{prf:example} (Square loss function)
:label: E1

When $\ell(y, z)=(y-z)^2$, the Bayes optimal predictor is given by

$$ f^*(x)=\mathbb{E}[y|\mathbf{x}=x] \tag{3}$$

```
````{prf:proof}

Instead of solving $(1)$ directly, note that we can solve it in a point-wise manner

$$
\begin{align*}
f^*(x) 
&= \underset{z\in \mathcal{Y}}{\text{argmin}}\; \mathbb{E}[(z-y)^2|\mathbf{x}=x]
\end{align*}
$$

The objective can be written as 

$$
\begin{align*}
\mathbb{E}[(z-y)^2|\mathbf{x}=x]
&= \mathbb{E}[(z- \mathbb{E}[y|\mathbf{x}=x] + \mathbb{E}[y|\mathbf{x}=x] - y)^2|\mathbf{x}=x]\\
&= \mathbb{E}[(z- \mathbb{E}[y|\mathbf{x}=x])^2|\mathbf{x}=x] + 2\mathbb{E}[(z- \mathbb{E}[y|\mathbf{x}=x])(\mathbb{E}[y|\mathbf{x}=x]-y)|\mathbf{x}=x] + \mathbb{E}[(\mathbb{E}[y|\mathbf{x}=x]-y)^2|\mathbf{x}=x]\\
&= \mathbb{E}[(z- \mathbb{E}[y|\mathbf{x}=x])^2|\mathbf{x}=x] + \mathbb{E}[(\mathbb{E}[y|\mathbf{x}=x]-y)^2|\mathbf{x}=x]\\
\end{align*}
$$

Since the last term in independent of $z$, it follows that the minimizer is given by $z=\mathbb{E}[Y|X=x]$, thereby proving the claim. This example tells us that for regression problems, the Bayes optimal predictor is simply given by the conditional expectation $(3)$. However, to compute $f^*(x)$, we need to know the conditional distribution $y|\mathbf{x}=x$, which is often unknown in practice. Consequently, this result has limited practical applicability.

````

Despite $(1)$ can be solved directly for certain family loss functions, in general, solving it is extremly challenging, for three reasons

1. The distributions $\mathcal{X}, \mathcal{Y}$ are typically unknown, and therefore we can't evaluate $(3)$ directly. And even if the distributions are known
2. The objective in $(1)$ involves optimizing over an integral.
$$ \mathbb{E}_{(\mathbf{x}, y)}[\ell(f(\mathbf{x}), y)] = \int \ell(f(\alpha), \beta) dp_{\mathbf{x}, y}(\alpha, \beta)$$
3. The set of feasible functions $f$ could be a large set.

In practice, we often cannot solve $(1)$ exactly. Therefore, we need ways to simplify $(1)$ into a tractable optimization problem.

## Improvement 1: Empirical risk minimization

Observe that while the distribution $\mathcal{X}, \mathcal{Y}$ are often unknown, we do have access to observations $\mathcal{D}=\{(\mathbf{x}_i, y_i)\}_{i\in [n]}$ sampled from the distribution. By invoking the law of large numbers, we can estimate the expectation by

$$\mathbb{E}_{(\mathbf{x}, y)}[\ell(f(\mathbf{x}), y)] \approx \frac{1}{n}\sum_{i=1}^n \ell(f(\mathbf{x}_i), y_i)$$

This motivates us to replace $(1)$ with a new optimization problem

$$\hat{f} =  \underset{f}{\text{argmin}}\; \underbrace{\frac{1}{n}\sum_{i=1}^n \ell(f(\mathbf{x}_i), y_i)}_{\hat{R}(f)} \tag{4}$$

The term $\hat{R}(f)$ is said to be the empirical risk (training loss) of $f$. It is an estimate of the true risk (test loss) $R(f)$. However, note that we are optimizing a different objective, which heavily depends on the training data $\mathcal{D}$ we used. A crucial point is that the minimizer $\hat{f}$ is not necessarily the minimizer of $(1)$. For example, we can easily achieve an empirical risk of $0$ by simply letting $\hat{f}$ interpolate the data. However, such a predictor may have a high true risk. Nevertheless, this reduction solves challenge 1 and 2 proposed in the previous section. 

## Improvement 2: Constrained optimization

A straightforward method to address challenge 3 is by constraining the set of feasible functions. Rather than optimizing over all measurable functions, we confine the feasible function to the set $\mathcal{F}$. This gives us the following

$$\hat{f}_{\mathcal{F}} = \underset{f\in \mathcal{F}}{\text{argmin}}\; \frac{1}{n}\sum_{i=1}^n \ell(f(\mathbf{x}_i), y_i) \tag{5}$$

Optimization problem $(5)$ is generally solvable if $\mathcal{F}$ is admits nice forms. If the functions in $\mathcal{F}$ is parameterized by $\theta\in \Theta$, then $(5)$ is equivalent to a parameter estimation problem

$$\hat{\theta} = \underset{\theta \in \Theta}{\text{argmin}}\; \frac{1}{n}\sum_{i=1}^n \ell(f_{\theta}(\mathbf{x}_i), y_i)$$

Which can be solved using algorithms like stochastic gradient descent. Such models are called a parametric model. If, on the other hand, $\mathcal{F}$ is not parameterized, then the resulting solution is called a nonparametric model. 

## Example: Linear regression

Suppose we restrict ourselves on the set of linear functions $\mathcal{F}=\{f:f(x)=\boldsymbol{\theta}^T\mathbf{x}\}$. Given observations $\mathcal{D}=\{(\mathbf{x}_i, y_i)\}_{i\in [n]}$, and using the square loss, the optimal parameter is given by 

$$\hat{\theta} = \underset{\theta}{\text{argmin}}\; \frac{1}{n}\sum_{i=1}^n (\boldsymbol{\theta}^T\mathbf{x}_i- y_i)^2$$

Letting $\mathbf{X}, \mathbf{y}$ denote the matrix of features/response, the above is equivalent to 

$$\hat{\boldsymbol{\theta}} = \underset{\boldsymbol{\theta}}{\text{argmin}}\; \frac{1}{n}||\mathbf{X}\boldsymbol{\theta}- \mathbf{y}||_2^2$$

This optimization problem can be solved using standard first order methods. From the first order condition

$$\nabla_{\boldsymbol{\theta}}(||\mathbf{X}\boldsymbol{\theta}- \mathbf{y}||_2^2) = 2(\mathbf{X}^T\mathbf{X})\boldsymbol{\theta} - 2(\mathbf{X}^T \mathbf{y}) = 0\implies \hat{\boldsymbol{\theta}} = (\mathbf{X}^T\mathbf{X})^{-1}(\mathbf{X}^Ty)$$

And this gives us the optimal predictor under empirical risk minimization. 